In [1]:
import torch
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import copy
import json
import numpy as np
import matplotlib.pyplot as plt 
import albumentations as A
from albumentations.pytorch import ToTensorV2
from models.efficientnet import EffNetModel
from pytorch_metric_learning import losses, miners, testers, samplers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
from tqdm import tqdm
import random
import wandb

In [2]:
from datasets_handlers.base_dataset import BaseImageDataset
from datasets_handlers.dataset_cub import cub200_dataset
from datasets_handlers.in_shop import in_shop_dataset
from datasets_handlers.stanford_online_products import stanford_products_dataset

In [3]:
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

In [4]:
stanford_train = stanford_products_dataset(split='train')
stanford_test = stanford_products_dataset(split='test')

In [5]:
#cub200_train = cub200_dataset(split='train')
#cub200_test = cub200_dataset(split='test')

In [6]:
#inshop_train = in_shop_dataset(split='train')
#inshop_gallery = in_shop_dataset(split='gallery')
#inshop_query = in_shop_dataset(split='query')

In [7]:
image_size = 224
train_transform = A.Compose(
    [
        #A.HorizontalFlip(p=0.5),
        #A.ImageCompression(quality_lower=99, quality_upper=100),
        #A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=10, border_mode=0, p=0.7),
        A.Resize(image_size, image_size),
        #A.CoarseDropout(max_height=int(image_size * 0.4), max_width=int(image_size * 0.4), max_holes=1, p=0.5),
        A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        ),
        ToTensorV2()
    ]
)

test_transform = A.Compose([
    A.Resize(image_size, image_size),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
    ToTensorV2()
])

In [8]:
dataset_train = BaseImageDataset(stanford_train, path_prefix='data/', augmentations=train_transform)
dataset_test = BaseImageDataset(stanford_test, path_prefix='data/', augmentations=test_transform)

In [9]:
parameters = {
    'dataset:': type(dataset_train.dataset).__name__,
    'n_epochs': 100,
    'batch_size': 64,
    'lr': 1e-5
}

In [10]:
wandb.init(project="my-test-project", entity="ilya_fedorov", config=parameters)

wandb: Currently logged in as: ilya_fedorov (use `wandb login --relogin` to force relogin)


In [11]:
train_targets = []
for x, target in stanford_train:
    train_targets.append(target)

In [12]:
loss_func = losses.TripletMarginLoss()

miner = miners.TripletMarginMiner(margin=0.05)

sampler = samplers.MPerClassSampler(
    train_targets, m=4, length_before_new_iter=len(dataset_train)
)

In [13]:
dataloader_train = DataLoader(
    dataset_train,
    batch_size=parameters['batch_size'],
    num_workers=8,
    pin_memory=True,
    sampler=sampler
)

In [14]:
model = EffNetModel('efficientnet-b0').cuda()
model = model.train()

Loaded pretrained weights for efficientnet-b0


In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=parameters['lr'])

In [16]:
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)

def test(train_set, test_set, model, accuracy_calculator):
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    test_embeddings, test_labels = get_all_embeddings(test_set, model)
    train_labels = train_labels.squeeze(1)
    test_labels = test_labels.squeeze(1)
    accuracies = accuracy_calculator.get_accuracy(
        test_embeddings, train_embeddings, test_labels, train_labels, True
    )
    return accuracies

In [17]:
accuracy_calculator = AccuracyCalculator(include=("mean_average_precision", "precision_at_1"), k=5)

In [ ]:
for epoch in range(parameters['n_epochs']):
    print(f'Starting epoch {epoch}')
    for data, labels in tqdm(dataloader_train):
        data = data.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()
        embeddings = model(data)
        #hard_pairs = miner(embeddings, labels)
        loss = loss_func(embeddings, labels)
        wandb.log({"train/loss": loss})
        loss.backward()
        optimizer.step()
        
    if epoch % 10 == 0:
        test_results = test(dataset_test, dataset_test, model, accuracy_calculator)
        wandb.log({"test/map@1": test_results['precision_at_1']})
        wandb.log({"test/map@5": test_results['mean_average_precision']})

Starting epoch 0


100%|███████████████████████████████████████| 1891/1891 [01:28<00:00, 21.40it/s]


Starting epoch 1


100%|█████████████████████████████████████████| 708/708 [01:46<00:00,  6.66it/s]


Starting epoch 2


...

100%|█████████████████████████████████████████| 708/708 [01:39<00:00,  7.09it/s]


Starting epoch 67


100%|█████████████████████████████████████████| 708/708 [01:37<00:00,  7.27it/s]


Starting epoch 68


 86%|███████████████████████████████████▍     | 612/708 [01:26<00:13,  6.94it/s]